In [1]:
import math
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
import pandas as pd
from keras.models import load_model
import numpy as np

Using TensorFlow backend.


In [ ]:
def create_dataset(dataset, window_size = 169):
    data_X, data_Y = [], []
    for i in range(len(dataset) - window_size - 1):
        a = dataset[i:(i + window_size), 0]
        data_X.append(a)
        data_Y.append(dataset[i + window_size, 0])
    return(np.array(data_X), np.array(data_Y))

In [ ]:
def fit_model(train_X, train_Y, window_size = 169):
    model = Sequential()
    
    model.add(LSTM(4, 
                   input_shape = (1, window_size)))
    model.add(Dense(1))
    model.compile(loss = "mean_squared_error", 
                  optimizer = "adam")
    model.fit(train_X, 
              train_Y, 
              epochs = 100, 
              batch_size = 1, 
              verbose = 2)
    
    return(model)

In [ ]:
rmse = []

for i in range(100):
    my_data = pd.DataFrame(new_cluster[i]).reset_index()
    data_raw = my_data['freq'].values.astype('float32').reshape(-1,1)
    scaler = MinMaxScaler(feature_range = (0, 1))
    dataset = scaler.fit_transform(data_raw)
    
    # Create test and training sets for one-step-ahead regression.
    window_size = 169
    test = dataset[-338:]
    train = dataset[:-338]
    train_X, train_Y = create_dataset(train, window_size)
    test_X, test_Y = create_dataset(test, window_size)
    print("Original training data shape:")
    print(train_X.shape)

    # Reshape the input data into appropriate form for Keras.
    train_X = np.reshape(train_X, (train_X.shape[0], 1, train_X.shape[1]))
    test_X = np.reshape(test_X, (test_X.shape[0], 1, test_X.shape[1]))
    print('model :'+str(i))
    print("New training data shape "+str(i)+':')
    print(train_X.shape)
    print('__________')
    
    model1 = fit_model(train_X, train_Y, window_size)
    
    model.save('my_model_'+str(i)+'.h5')  # creates a HDF5 file 'my_model.h5'
    
    rmse_train, train_predict = predict_and_score(model1, train_X, train_Y)
    rmse_test, test_predict = predict_and_score(model1, test_X, test_Y)
    
    rmse.append(rmse_test)
    
    print("Training data score: %.2f RMSE" % rmse_train)
    print("Test data score: %.2f RMSE" % rmse_test)
